## Prejudice Remover Regularizer for Images

1. [Introduction](#1.-Introduction)
2. [Data preparation](#2.-Data-preparation)
3. [Classifier network](#3.-Classifier-network)
	*  [Model Fairness for Classifier Network](#Model-Fairness-for-Classifier)
4. [Prejudice Remover Regularizer](#4.-Prejudice-Remover-Regularizer)
	* [Model Fairness: with PRR](#Model-Fairness:-with-PRR)
5. [Summary](#5.-Summary)
6. [References](#References)

### 1. Introduction
Welcome !

We hope you have had a chance to go through previous tutorials on fairness in AI by Sony. In one of the tutorials, we applied `Prejudice Remover Regularizer (hereafter referred to as PRR)` technique to train a fair classifier on UCI Adult (Census) Dataset. As a result, model fairness improved without causing much drop in accuracy.

In this tutorial, we will explore PRR for visual recognition task on CelebA dataset (real world image dataset).

Before we go into a detailed explanation of the PRR training procedure, here is a sneak peek into the steps involved in the process:

### Preparation
Let's start by installing nnabla and accessing [nnabla-examples repository](https://github.com/sony/nnabla-examples). If you're running on Google Colab, make sure that your Runtime setting is set as GPU, which can be set up from the top menu (Runtime → change runtime type), and make sure to click **Connect** on the top right-hand side of the screen before you start.

In [ ]:
# Preparation
# May show warnings for newly imported packages if run in Colab default python environment.
# Please click the `RESTART RUNTIME` to run the following script correctly.
# The error message of conflicts is acceptable.
!git clone https://github.com/sony/nnabla-examples.git
!pip install albumentations
!pip install nnabla-ext-cuda116

In [ ]:
%cd nnabla-examples/responsible_ai/prejudice_remover_regularizer_images

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('images/Prejudice_Remover_Regularizer_workflow_diagram.png')
cv2_imshow(img)

As illustrated in the picture, in the first step we will train a classifier model to make predictions & analyze model fairness. Then the subsequent step shows how PRR training can be employed to make the model fair if the fairness metric is not satisfactory.

At first, all we need is to import all the necessary python libraries.

In [ ]:
import os
import glob
import pickle
import shutil
import numpy as np
from PIL import Image
import albumentations as A
import nnabla as nn
from nnabla.utils.data_iterator import data_iterator_simple
import classifier as clf
from utils import utils

Let us train an `Attractive` classifier that is not dependent on gender expression. 

### 2. Data preparation

Before training a classifier model, let's download and split the CelebA dataset into three categories: training, validation and test sets.

In [ ]:
# download the celeba dataset and unzip
URL = "https://www.dropbox.com/s/d1kjpkqklf0uw77/celeba.zip?dl=0"
ZIP_FILE= "./data/celeba.zip"
!mkdir -p ./data/
!wget -N $URL -O $ZIP_FILE
!unzip $ZIP_FILE -d ./data/
!rm $ZIP_FILE

In [ ]:
def split_celeba_dataset(img_path, attr_path, out_dir, split="test"):
    
    """
    split the celebA dataset
    Args:
        img_path (str): image path directory 
        attr_path (str): celebA attribute file path (ex: list_attr_celeba.txt)
        out_dir (str): Path where the split data to be saved
        split (string): split the dataset depends on the split attribute(train, valid, and test)
    """
    # as per the author's remark, we split the dataset
    train_beg = 0  # train starts from
    valid_beg = 162770  # valid starts from
    test_beg = 182610  # test starts from
    
    label_file = open(attr_path, 'r')
    label_file = label_file.readlines()
    
    # skipping the first two rows for header
    total_samples = len(label_file) - 2
    if split == 'train':
        number_samples = valid_beg - train_beg
        beg = train_beg
    
    elif split == 'valid':
        number_samples = test_beg - valid_beg
        beg = valid_beg
    
    elif split == 'test':
        number_samples = total_samples - test_beg
        beg = test_beg
    else:
        print('Error')
        return
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    for i in range(beg + 2, beg + number_samples + 2):
        temp = label_file[i].strip().split()
        src_dir = os.path.join(img_path,temp[0])
        dst_dir = os.path.join(out_dir,temp[0])
        shutil.copy(src_dir, dst_dir)
    print("splitting completed")

[CelebA](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) is a dataset with 2,022,599 celebrity face images, each with 40 binary attributes labels. 

Now let us train an `Attractive` classifier that is not dependent on gender expression. `Male` attribute corresponds to gender expression and the target attribute is `Attractive`.

In [ ]:
def data_iterator_celeba(img_path, attr_path, batch_size,
                         target_attribute ='Attractive', protected_attribute = 'Male', 
                         num_samples=-1, augment=False, shuffle=False, rng=None):
    """
    create celebA data iterator
    Args:
        img_path (str) : image path directory
        attr_path (str) : celebA attribute file path (ex: list_attr_celeba.txt)
        batch_size (int) :  number of samples contained in each generated batch
        target_attribute (str) : target attribute (ex: Arched EyeBrows, Bushy Eyebrows, smilling,etc..)
        protected_attribute (str): protected attribute (ex: Male, Pale_Skin)
        num_samples (int) : number of samples taken in data loader
                            (if num_samples=-1, it will take all the images in the dataset)
        augment (bool) : data augmentation (True for training)
        shuffle (bool) : shuffle the data (True /False)
        rng : None
    Returns:
        simple data iterator
    """

    imgs = []
    for file in sorted(os.listdir(img_path), key=lambda x: int(x.split(".")[0])):
        imgs.append(os.path.join(img_path,file))
    with open(attr_path, 'r') as f:
        lines = f.readlines()

    attr_list = lines[1].strip().split()
    attr_idx_dict = {attr: i for i, attr in enumerate(attr_list)}
    labels_dict = {}
    for line in lines[2:]:
        line = line.strip().split()
        key = line[0]
        attr = line[1:]
        labels_dict[key] = np.array([int((int(attr[attr_idx_dict[target_attribute]]) + 1) / 2), int((int(attr[attr_idx_dict[protected_attribute]]) + 1) / 2)])

    # as per the author's citation, we have transformed the input image
    # (resize to , 256×256, 224×224)
    pre_process = [(256, 256), (224, 224)]
    mean_normalize = (0.485, 0.456, 0.406)
    std_normalize = (0.229, 0.224, 0.225)

    if augment:
        transform = A.Compose([
            A.Resize(pre_process[0][0], pre_process[0][1]),
            A.RandomCrop(width=pre_process[1][0], height=pre_process[1][1]),
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=mean_normalize, std=std_normalize)
        ])
    else:
        transform = A.Compose([
            A.Resize(pre_process[0][0], pre_process[0][1]),
            A.CenterCrop(width=pre_process[1][0], height=pre_process[1][1]),
            A.Normalize(mean=mean_normalize, std=std_normalize)
        ])
    if num_samples == -1:
        num_samples = len(imgs)
    else:
        print("Num. of data ({}) is used for debugging".format(num_samples))

    def load_func(i):
        img = Image.open(imgs[i])
        img = np.array(img.convert('RGB'))
        # transform
        transformed_image = transform(image=img)['image'].transpose(2, 0, 1)
        return transformed_image, labels_dict[os.path.basename(imgs[i])]

    return data_iterator_simple(load_func, num_samples, batch_size, shuffle=shuffle, rng=rng, with_file_cache=False)

Let’s start with importing basic modules to switch between CPU and GPU.

In [ ]:
from nnabla.ext_utils import get_extension_context
context = "cudnn" # for cpu set context as 'cpu'
device_id = 0
ctx = get_extension_context(context, device_id=device_id)
nn.set_default_context(ctx)

### 3. Classifier network
To train a classifier, we have taken [ResNet-50](https://nnabla.org/pretrained-models/nnp_models/imagenet/Resnet-50/Resnet-50.nnp) pretrained on [ImageNet](https://image-net.org/) as the base architecture. Fully connected layer in ResNet is replaced with two fully connected layers of size 2048. Dropout and ReLU are applied. We train all models with sigmoid cross-entropy loss for 20 epochs with a batch size of 32. We use the [Adam](https://arxiv.org/abs/1412.6980) optimizer with a learning rate of 1e-3.

We have trained the Attribute Classifier and saved the model with the best accuracy on the validation set. Now let us get these pre-trained classifier weights and evaluate model fairness. 

PS:
If you want to train the Attribute Classifier from scratch, please refer to our GitHub page and follow the steps.

In [ ]:
# download the pre-trained weights
!wget https://nnabla.org/pretrained-models/nnabla-examples/responsible_ai/prejudice_remover_regularizer_images/best_baseline.h5
!wget https://nnabla.org/pretrained-models/nnabla-examples/responsible_ai/prejudice_remover_regularizer_images/val_baseline.pkl

In [ ]:
nn.clear_parameters()

attribute_classifier_model = clf.AttributeClassifier(model_load_path=r'./best_baseline.h5')
# split the dataset
split_celeba_dataset(r'./data/celeba/images', r'./data/celeba/list_attr_celeba.txt', r'./test',split="test")
# load dataloader
test = data_iterator_celeba(img_path= r'./test',
                            attr_path= r'./data/celeba/list_attr_celeba.txt',
                            target_attribute ='Attractive', protected_attribute = 'Male',
                            batch_size=32)
cal_thresh = pickle.load(open(r'./val_baseline.pkl', 'rb'))['cal_thresh']

### Model Fairness for Classifier

Let's start our investigation of classifier model fairness by analyzing the predictions made on the test set. In this tutorial, we use the Average Precision (AP) metric to measure classifier accuracy & three metrics to measure model fairness: Calders-Verwer score (CV score), Difference in Equality of Opportunity (DEO) and Bias Amplification (BA). CV Score is the absolute difference between conditional probabilities of the positive class for protected attributes. [DEO](https://arxiv.org/abs/2004.01355) is the absolute difference in False Negative Rate (FNR) for the protected attribute group. [BA](https://arxiv.org/abs/2102.12594) is a metric proposed by Wang and Russakovsky. Intuitively, BA measures how much more often a target attribute is predicted with a protected attribute than the ground truth value.

In [ ]:
test_targets, test_scores = attribute_classifier_model.get_scores(test)
test_pred = np.where(test_scores > cal_thresh, 1, 0)
outf = test_scores[test_targets[:, 1] == 0]
outm = test_scores[test_targets[:, 1] == 1]
ap = utils.get_average_precision(test_targets[:, 0], test_scores)
cv_score = utils.get_cvs(outf,outm,cal_thresh)
deo = utils.get_diff_in_equal_opportunity(test_targets[:, 1],
                                              test_targets[:, 0], test_pred)
ba = utils.get_bias_amplification(test_targets[:, 1],
                                     test_targets[:, 0], test_pred)

# plot the fairness
utils.plot_fairness_multi(deo,cv_score,ba,ap,"Baseline")
print('Test results: ')
print('AP : {:.1f}', 100 * ap)
print('DEO : {:.1f}', 100 * deo)
print('BA : {:.1f}', 100 * ba)
print('CV : {:.1f}', 100 * cv_score)



As seen above, predictions are definitely not fair when considered in the context of `sex` as a sensitive attribute.

Now let's mitigate the bias using `Prejudice Remover Regularizer'

### 4. Prejudice Remover Regularizer

Before going to the Prejudice Remover Regularizer method, let's understand what is prejudice and different prejudice methods.

**Prejudice:** Prejudice means a statistical dependence between a sensitive variable S, and the target variable, Y, or a non-sensitive variable, X. There are three types of prejudices: `direct prejudice`, `indirect prejudice`, and `latent prejudice`

1. Direct prejudice: The prediction model directly depends on S
2. Indirect prejudice: Statistical dependence of Y on S, even lack of direct S
3. Latent prejudice: Statistical dependence of X on S

In this tutorial we discuss how to reduce Indirect prejudice, using the Prejudice Removal Regularization Technique. We next show an index to quantify the degree of indirect prejudice, which is defined as the mutual information between Y and S.
$$PI =\sum_{y,s∈D}\hat{Pr}[y,s] ln\frac{\hat{Pr}[y, s]}{\hat{Pr}[y]\hat{Pr}[s]}$$, we refer this index as a `prejudice index`(PI for short).

In this PRR technique, there are two types of regularizers. PRR technique involves adding two regularizer terms to cost function: L2 regularizer, $||θ||^2$, to reduce over-fitting and Prejudice remover regularizer, $R(D,θ)$, to enforce fair classification. So, the objective function to minimize is rewritten as :

In [ ]:
img = cv2.imread('images/Prejudice_Remover_Regularizer_Equation.png')
cv2_imshow(img)

where λ and η are positive regularization parameters.

in the above equation prejudice remover regularizer:
* Directly tries to reduce the prejudice index(PI).
* Smaller value more strongly constraints independence b/w Y and S

in short :

PRLR : Loss function(log-likelihood) + Prejudice Remover Regularizer(mutual information) + l2 Regularizer

We have trained the PRR model and saved the model with the best accuracy on the validation set. If you want to train the PRR model from the scratch please refer to our GitHub page and follow the steps.

Now let us get the pre-trained weights for the PRR and load the model. The supplied λ and η values, that tune fairness versus accuracy, are set to λ = 1e-05 and η = 2 (If η is 0, the network behaves as a simple base class classifier network). We heuristically found that these settings result in a balanced increase of the fairness value during training. You may train with different regularization parameter values and check the impact of different λ and η values on model performance and fairness.

Let's check the model fairness after PRR training

In [ ]:
!wget https://nnabla.org/pretrained-models/nnabla-examples/responsible_ai/prejudice_remover_regularizer_images/best_prr.h5
!wget https://nnabla.org/pretrained-models/nnabla-examples/responsible_ai/prejudice_remover_regularizer_images/val_prr.pkl

In [ ]:
nn.clear_parameters()

attribute_classifier_model = clf.AttributeClassifier(model_load_path=r'./best_prr.h5')
# split the dataset
# split_celeba_dataset(r'./data/celeba/images', r'./data/celeba/list_attr_celeba.txt', r'./test',split="test")
# load dataloader
test = data_iterator_celeba(img_path= r'./test',
                            attr_path= r'./data/celeba/list_attr_celeba.txt',
                            target_attribute ='Attractive', protected_attribute = 'Male',
                            batch_size=32)
cal_thresh = pickle.load(open(r'./val_prr.pkl', 'rb'))['cal_thresh']

### Model Fairness: with PRR

In [ ]:
test_targets, test_scores = attribute_classifier_model.get_scores(test)
test_pred = np.where(test_scores > cal_thresh, 1, 0)
outf = test_scores[test_targets[:, 1] == 0]
outm = test_scores[test_targets[:, 1] == 1]
ap = utils.get_average_precision(test_targets[:, 0], test_scores)

cv_score = utils.get_cvs(outf,outm,cal_thresh)

deo = utils.get_diff_in_equal_opportunity(test_targets[:, 1],
                                                     test_targets[:, 0], test_pred)
ba = utils.get_bias_amplification(test_targets[:, 1],
                                     test_targets[:, 0], test_pred)

utils.plot_fairness_multi(deo,cv_score,ba,ap,"PRR")
print('Test results: ')
print('AP : {:.1f}', 100 * ap)
print('DEO : {:.1f}', 100 * deo)
print('BA : {:.1f}', 100 * ba)
print('CV : {:.1f}', 100 * cv_score)

Plots above show how model fairness improved after induction of PRR into training. DEO & CV score have improved compared to the baseline model. After PRR training, classification accuracy has dropped. But, it can be controlled. Ideally, user must take a call on acceptable amount of trade off between accuracy and fairness. 

### 5. Summary
 
In this tutorial, we have demonstrated how to reduce indirect prejudice using `Prejudice Remover Regularizer` procedure in a visual recognition task.
Also, note that it is not possible to optimize a model for all the fairness metrics in real-time: to read more about this, explore the [Impossibility Theorem of Machine Fairness](https://arxiv.org/abs/2007.06024).

Also, making fair predictions comes at a cost: sometimes it will reduce the performance [AP] of our model (hopefully, only little). However, in many cases, this would be a small price to pay.

### 6. References

1. "Fairness-aware classifier with prejudice remover regularizer". Toshihiro Kamishima, Shotaro Akaho, Hideki Asoh & Jun Sakuma. Joint European Conference on Machine Learning and Knowledge Discovery in Databases ECML PKDD 2012: Machine Learning and Knowledge Discovery in Databases pp 35–50.
2. "Equality of opportunity in supervised learning". Hardt, Moritz, Eric Price, and Nati Srebro.Advances in neural information processing systems 29 (2016)
3. "Directional bias amplification". Wang, Angelina, and Olga Russakovsky. International Conference on Machine Learning. PMLR, 2021.
4. "The Impossibility Theorem of Machine Fairness--A Causal Perspective".Saravanakumar, Kailash Karthik. arXiv preprint arXiv:2007.06024 (2020).
5. "Adam: A method for stochastic optimization". Kingma, Diederik P., and Jimmy Ba. arXiv preprint arXiv:1412.6980 (2014).
6. "Large-scale celebfaces attributes (celeba) dataset". Liu, Ziwei, Ping Luo, Xiaogang Wang, and Xiaoou Tang.  Retrieved August 15, no. 2018 (2018): 11.
